# random

Picks a number of points from the raster image and the same number from `intersections` at random. Uses `np.linalg.lstsq` to fit. Runs a bunch of times and keeps the one with the lowest RMSE.

In [2]:
import numpy as np

rng = np.random.default_rng()

points = np.loadtxt('/Users/kmarsh/Code/kmarsh/autogeoreferencer/data/OH-Champaign-72-01.csv', delimiter=',', usecols=(0, 1))
intersections = np.loadtxt('/Users/kmarsh/Code/kmarsh/autogeoreferencer/data/39021-intersections-3857.psv', delimiter='|')

pts = 6
primary = rng.choice(points, pts)

n = primary.shape[0]
pad = lambda x: np.hstack([x, np.ones((x.shape[0], 1))])
unpad = lambda x: x[:,:-1]

best_score = None
best_transform = None

for i in range(1_000_000):
  secondary = rng.choice(intersections, pts)

  X = pad(primary)
  Y = pad(secondary)

  # Solve the least squares problem X * A = Y
  # to find our transformation matrix A
  A, res, rank, s = np.linalg.lstsq(X, Y)

  transform = lambda x: unpad(np.dot(pad(x), A))

  # max_error = np.abs(secondary - transform(primary)).max()
  max_error = np.sqrt(np.mean((secondary - transform(primary))**2))

  if best_score is None:
    best_score = max_error

  if max_error <= best_score:
    best_score = max_error
    best_transform = A

print(best_score)

<ipython-input-2-56c4fe649168>:26: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  A, res, rank, s = np.linalg.lstsq(X, Y)


755.1512428230834


In [5]:
A = best_transform
transform = lambda x: unpad(np.dot(pad(x), A))

transform(np.asarray([[2126, 598], [588, 1086]]))

array([[-9299810.34252691,  4884979.52299427],
       [-9334239.91174546,  4880116.69051279]])